In [ ]:
# Import Libraries & Authenticate GEE

import ee
ee.Authenticate()
ee.Initialize(project='nitk25')

In [ ]:
import geemap

Map = geemap.Map()

point = ee.Geometry.Point([74.85, 12.95])  # Karnataka region
basin = point.buffer(20000)  # 20 km buffer

Map.centerObject(basin, 9)
Map.addLayer(basin, {}, "Watershed")
Map


In [ ]:
rain = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
        .filterDate("2020-01-01", "2020-12-31") \
        .filterBounds(basin) \
        .sum()

Map.addLayer(rain, {"min":0, "max":2000}, "Rainfall")


In [ ]:
lulc = ee.Image("ESA/WorldCover/v100/2020")

Map.addLayer(lulc, {}, "LULC")


In [ ]:
dem = ee.Image("USGS/SRTMGL1_003")

Map.addLayer(dem, {"min":0, "max":3000}, "DEM")


In [ ]:
stack = rain.addBands(lulc).addBands(dem)


In [ ]:
samples = stack.sample(
    region=basin,
    scale=1000,
    numPixels=1000,
    geometries=False
)

import pandas as pd
df = geemap.ee_to_df(samples)
df.head()


In [ ]:
df["Runoff"] = 0.3 * df["precipitation"]   # simple assumption


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

X = df[["precipitation", "Map", "elevation"]]
y = df["Runoff"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("R² Score:", r2_score(y_test, y_pred))


In [ ]:
model.feature_importances_


In [ ]:
# Create comparison dataframe
comparison = X_test.copy()
comparison["Actual_Runoff"] = y_test.values
comparison["Predicted_Runoff"] = y_pred

# Show first 10 samples
comparison.head(10)


In [ ]:
comparison["Error"] = comparison["Actual_Runoff"] - comparison["Predicted_Runoff"]
comparison["Absolute_Error"] = abs(comparison["Error"])

comparison.head()


In [ ]:
comparison["Absolute_Error"].mean()


In [ ]:
import matplotlib.pyplot as plt

plt.scatter(comparison["Actual_Runoff"], comparison["Predicted_Runoff"])
plt.xlabel("Actual Runoff")
plt.ylabel("Predicted Runoff")
plt.title("Actual vs Predicted Runoff")
plt.show()


In [ ]:
comparison.sample(5)


In [ ]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)
